In [5]:
# INITS + IMPORTS
from typing import List
from openai import OpenAI
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv

BASE_URL = "http://199.94.61.113:8000/v1/"
API_KEY=api_key="yen.k@northeastern.edu:79wlaJxIMpzvkt61S8Xi"
assert BASE_URL is not None
assert API_KEY is not None

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

load_dotenv() 

# Set up the client id and client secret (you can find the client secret 
# from the spotify dev page)
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
REDIRECT_URI = 'http://localhost:8888/callback'
SCOPE = 'playlist-modify-public user-modify-playback-state user-top-read'

# Create a spotify object passing the client id, client secret, 
# redirct url (which doesn't matter, just set it as your local host
# as shown below), and scope
spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE,
                                               cache_path=".cache-<your-username>"))

# TODO define the data classes

In [37]:
class SpotifyAgent:
    conversation: List[dict]
    client: OpenAI
    spotify: SpotifyOAuth
    playlist_id: str
    seed_artists: List[str]
    seed_tracks: List[str]
    seed_genres: List[str]
    # current_mood: int # TODO: figure this out

    def create_playlist(self):
        user_id = self.spotify.current_user()['id']  
        playlist_name = 'Spotify Chatbot Session'
        playlist_description = 'A playlist created by the Spotify Chatbot.'
        # Get the current user's playlists
        playlists = self.spotify.current_user_playlists()

        # Look for the playlist by name
        existing_playlist = None
        for playlist in playlists['items']:
            if playlist['name'] == playlist_name:
                existing_playlist = playlist
                break

        if existing_playlist:
            # If the playlist exists, take action: delete all tracks from it
            playlist_id = existing_playlist['id']
            print(f"Found existing playlist '{playlist_name}', clearing tracks...")
            
            # Get all tracks
            track_uris = []
            results = self.spotify.playlist_items(playlist_id)
            track_uris.extend([item['track']['uri'] for item in results['items']])

            # Remove all tracks
            if track_uris:
                self.spotify.playlist_remove_all_occurrences_of_items(playlist_id, track_uris)
                print(f"All tracks removed from playlist '{playlist_name}'.")

            return playlist_id
        else:
            # If the playlist does not exist, create a new one
            print(f"Playlist '{playlist_name}' does not exist. Creating a new one...")
            res = self.spotify.user_playlist_create(user=user_id, 
                                    name=playlist_name, 
                                    public=True, 
                                    description=playlist_description)
            print(f"Playlist '{playlist_name}' created successfully.")
            return res['id']

    # Add one song or multiple songs
    def add_tracks_to_playlist(self, tracks):
        track_uris = []
        for track in tracks:
            track_uris.append(track['uri'])
        spotify.playlist_add_items(self.playlist_id, track_uris)

    def get_recommendations(self):
        limit = 10
        # if all seeds are none, return top songs
        if len(self.seed_artists) == 0 and len(self.seed_tracks) == 0 and len(self.seed_genres) == 0:
            print("No seeds provided. Returning top tracks.")
            return spotify.current_user_top_tracks(limit=limit, offset=0, time_range='short_term')['items']
        
        recommendations = spotify.recommendations(
            seed_artists=self.seed_artists,
            seed_tracks=self.seed_tracks,
            seed_genres=self.seed_genres,
            limit=limit
        )

        return recommendations['tracks']
    
    def add_to_seed_artists(self, artist_name):
        # Perform a search query for the artist
        results = spotify.search(q='artist:' + artist_name, type='artist')
        
        items = results['artists']['items']
        if items:
            artist = items[0]
            self.seed_artists.append(artist['uri']) # add first artist
        
         # TODO Figure out error handling

    def add_to_seed_tracks(self, track_name):
        results = spotify.search(q='track:' + track_name, type='track')
        items = results['tracks']['items']
        if items:
            track = items[0]
            self.seed_tracks.append(track['uri'])
    
    def add_to_seed_genres(self, genre_name):
        # TODO if genere exists add...
        self.seed_genres.append(genre_name)

    
    def __init__(self, client: OpenAI, spotify: SpotifyOAuth):
        self.client = client
        self.spotify = spotify
        self.playlist_id = self.create_playlist() # Create a playlist for the session
        self.conversation = []
        self.seed_artists = []
        self.seed_tracks = []
        self.seed_genres = []
        # self.current_mood = 0

In [38]:
agent = SpotifyAgent(client, spotify)
# res = agent.get_recommendations()
# agent.add_tracks_to_playlist(res)

agent.add_to_seed_artists("Ariana Grande")
print(agent.seed_artists)
agent.add_to_seed_tracks("Shake it off")
print(agent.seed_tracks)
res = agent.get_recommendations()
for track in res:
    print(track['name'])
# figuring out adding specific songs


Found existing playlist 'Spotify Chatbot Session', clearing tracks...
['spotify:artist:66CXWjxzNUsdJxJ2JdwvnR']
['spotify:track:3pv7Q5v2dpdefwdWIvE7yH']
Dance Alone
Learning To Fly
Sit Still, Look Pretty
Luxurious
Coincidence
Finesse - Remix; feat. Cardi B
enough for you
Walk the Dinosaur - From "Ice Age: Dawn of the Dinosaurs"
Chandelier
London Boy
